# Recurrent Neural Networks



## Recurrence

### Simple concept idea
Recurrent neural networks models are linked to time and sequencial problems.

Perhaps the most simple recurrent network is the Elman Neural Network [2] illustrated in Figure 1.

![Elman Net](./images/srn2.png "Figure 1")

The Elman Net can be denoted as Simple Recurrent Network (SRN) and can be described mathematically [2] as:

$$
\begin{aligned}
x(t) & =w(t)+s(t-1) \\
s_j(t) & =f\left(\sum_i x_i(t) u_{j i}\right) \\
y_k(t) & =g\left(\sum_j s_j(t) v_{k j}\right)
\end{aligned}
$$
The sigmoid activation function:
$$
f(z)=\frac{1}{1+e^{-z}}
$$

where:

- $x(t)$ is the input at time $t$ formed by concatenating:
    - $w(t)$ represents the current word;
    - $s(t-1)$ is the output from neurons in context layer $s$ at time $t-1$;
- $s_j(t)$ is the context output at time $t$ with $j$ (#hidden neurons);
- $y_k(t)$ is the output at time $t$ with $k$ (output dim);
- $u_{ij}$ represents the input weights with $i$ (input dim);
- $v_{kj}$ represents the output weights bias;

I think this representation is very similar to the representation shown in [mit-lecture2-RNNs](https://www.youtube.com/watch?v=ySEx_Bqxvvo&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI&index=3) with recurrent cell and unfold illustration through time.

In an attempt to understand basic fundamentals of recurrent nets, I created a network from scratch to try to solve the xor problem as a sequence as proposed by Elman [3].

### The sequence XOR Problem
Input : 1010001110101 ...

Output: 011100100111? ...

When the network has received the first bit-1 in the example above-there is a 50% chance that the next bit will be a 1 (or a 0). When the network receives the second bit (0),
however, it should then be possible to predict that the third will be the XOR, 1.

Dataset

In [27]:
import torch

# Reproducibility
torch.manual_seed(13)

# Create output XOR sequence
def xor(w):
    # The first element of sequence has 50% of chance of happens
    y = torch.randint(0, 2, (1,))
    
    # The next elements consider two time steps
    for i in range(len(w)-1):
        y = torch.cat((y, w[i] ^ w[i+1]), 0) # xor sequence

    # Add batch dimension 
    y = y.view(y.shape[0], 1)
    
    return y

# Define random sequence input
n_samples = 300
w = torch.randint(0, 2, (n_samples, 1,))

# Get target output
l = xor(w)

# Concatenate the data
samples = list(zip(w.float(), l.float()))

Build

In [28]:
# For debug, uncomment to track wich node autograd has problem 
#torch.autograd.set_detect_anomaly(True)

class SRN(torch.nn.Module):
    def __init__(self, w_dim, h_dim):
        super().__init__()
        self.hx = torch.nn.Linear(w_dim + h_dim, h_dim)  # Hidden layer with input (w) + context units (s-1)        
        self.ho = torch.nn.Linear(h_dim, 1)
        self.s_ = torch.rand(h_dim) # Random initialize context units, hence: 
                                    # for t=0 and h_dim=2 -> s(t-1) = [s0(-1), s1(-1)]
        # Activation function
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, w): 
        e = torch.cat((w, self.s_)) # w(t) + s(t-1)
        s = self.sigmoid(self.hx(e)) # s(t)
        o = self.ho(s)               # logits
        y = self.sigmoid(o)         # probs
        
        self.s_ = s.detach()        # old context
        
        return y

Train

In [29]:
h_dim = 4 # Number of neurons in hidden layer

model = SRN(1, h_dim) # The configuration of SRN is in:hn-cu:out (1:44:1)

print(model)

w = w.float().requires_grad_(True)
l = l.float().requires_grad_(True)

epochs = 100

optim = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

bce = torch.nn.BCELoss()

for epoch in range(epochs):
    rl = 0.0
    i = 0
    for f, t in samples:
        optim.zero_grad()
        
        p = model(f)
        
        loss = bce(p, t)
        
        loss.backward()

        optim.step()
        
        rl += loss.item()
    
    # Handcraft learnnig rate decay
    if epoch % 20 == 0 and epoch > 0:
        for param_group in optim.param_groups:
            print(f"Changing learning rate to {param_group['lr']/2}")
            param_group['lr'] = param_group['lr']/2
    
    rl /= n_samples
    
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, rl))
        
print("Finished training")

SRN(
  (hx): Linear(in_features=5, out_features=4, bias=True)
  (ho): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	Training Loss: 0.754951
Epoch: 2 	Training Loss: 0.741379
Epoch: 3 	Training Loss: 0.737758
Epoch: 4 	Training Loss: 0.732614
Epoch: 5 	Training Loss: 0.717725
Epoch: 6 	Training Loss: 0.687708
Epoch: 7 	Training Loss: 0.673383
Epoch: 8 	Training Loss: 0.662864
Epoch: 9 	Training Loss: 0.608036
Epoch: 10 	Training Loss: 0.550075
Epoch: 11 	Training Loss: 0.464484
Epoch: 12 	Training Loss: 0.449802
Epoch: 13 	Training Loss: 0.465556
Epoch: 14 	Training Loss: 0.403875
Epoch: 15 	Training Loss: 0.490509
Epoch: 16 	Training Loss: 0.457623
Epoch: 17 	Training Loss: 0.446882
Epoch: 18 	Training Loss: 0.435092
Epoch: 19 	Training Loss: 0.436977
Epoch: 20 	Training Loss: 0.437063
Changing learning rate to 0.05
Epoch: 21 	Training Loss: 0.411269
Epoch: 22 	Training Loss: 0.484933
Epoch: 23 	Training Loss: 0.441611
Epoch: 24 	Training Loss: 0.3

Inference/Evaluation

In [31]:
# Input test sequence
wtest = torch.randint(0, 2, (100, 1,))

# output test sequence
ltest = xor(wtest)

test_samples = list(zip(wtest.float(), ltest.float()))
c = 0
for f, t in test_samples:
    w = round(f.item())
    y = round(model(f).item())
    t = round(t.item())
    
    #print(f"w: {w}, y:{y}, t:{t}")
    
    if y == t:
        c += 1

print("acc:", end= '')        
print(c/len(wtest))

acc:0.92


It looks like some learning has happened.... but I stopped here because the purpose of this task was just to understand recurrence in a simple example using an SRN. 
 
More complex models can be implemented using recurrent layers such as [nn.RNN, nn.LSTM, nn.GRU](https://pytorch.org/docs/stable/nn.html) using pytorch or other frameworks.

The next steps will focus on understanding the fundamentals of transformer networks with simple examples.

Reference:
1. https://www.youtube.com/watch?v=ySEx_Bqxvvo&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI&index=2
2. https://github.com/ramonlins/obsidian/tree/feature/dl/rnn/Papers/recurrent%20networks/Tomas%20Mikolov
3. https://github.com/ramonlins/obsidian/blob/feature/dl/rnn/Papers/recurrent%20networks/Tomas%20Mikolov/references/elman1990.pdf
4. https://github.com/karpathy/nn-zero-to-hero

Additional Materials:

https://pabloinsente.github.io/the-recurrent-net